# Reading, and tyding data
In this notebook the data is loaded, explored,  and preprocessed.

In [ ]:
# Import packages
import pandas as pd

# graphics
import matplotlib.pyplot as plt
import seaborn as sns

# text procesing
import re
from datetime import datetime
import numpy as np

In [ ]:
# Movies: 
mov_raw = pd.read_csv('../data/raw/ml-latest-small/movies.csv', encoding='utf-8')
# Users' ratings
ratings_raw = pd.read_csv('../data/raw/ml-latest-small/ratings.csv', encoding='utf-8')
# Tags: Xxx, abc
tags_raw = pd.read_csv('../data/raw/ml-latest-small/tags.csv', encoding='utf-8')

## Exploring the Data
Here I plan several hipotesis that represent new features:
+ Encode each genre and assigna weight to it. Maybe some genres have higher ratings overall

+ Assign a weight to each movie based on the number of views it has. Maybe the most viewed movies have a higher chance to receive a higher rating from a new user.

+ Maybe the ratings given by the users should be weighetd based on the number of movies watched by the user. 

In [ ]:
# Exploring the data
# Try to find an encode for each genre and assign a weight to it. 
#   Then predict the ratng based on the weights of the genres

# Assign a weight to each movie that represents how many users viewed it. 
#   MAybe the most viewed movies have a better chance to be a great recommendation
#   to new users

# The more movies a user watchs the more valuable their ratings, maybe.

# 
pro_movies

In [ ]:
# Extract the year from the name of the move (if exists)
# dtf_products["name"] = 
#   dtf_products["title"].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x).strip())

# For the Movie Dataframe:

# copy raw dataframe:
pro_movies = mov_raw.copy()

# Extract the Year from the Title of the Movie (if its between parenthesis)
pro_movies['Year'] = pro_movies['title'].apply(
    lambda x: int(x.split("(")[-1][:4].replace(")", "").strip()) # if there are 2 years (like 2006-2010), the first year is taken
        if "(" in x else np.nan)    # if theres a ( in the Name, set the year, else, a NA
        
pro_movies['title'] = pro_movies['title'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x).strip())


# copy raw ratings data:
pro_ratings = ratings_raw.copy()

# Add date column
pro_ratings['Date'] = pro_ratings['timestamp']


In [ ]:
# One-hot encoding of genres: 

# First get a list of lists (each list is the list of genres for each movie)
aux = [i.split('|') for i in pro_movies.genres.unique()]
# Then create a set (unique array of elements) and remove the no genres listed
vocab = list(set(i for k in aux for i in k))
vocab.remove('(no genres listed)')
print("Genres present in the dataset: ", vocab)

# Now, create a column for each genre:
for genre in vocab:
    pro_movies[genre] = pro_movies.genres.apply(lambda x: 1 if genre in x else 0)

pro_movies

Onehot encoding, in this case, is quite bad, as the dataframe shows below, theres very sparse. This means that our model will have a lot of entries that will, most of the time, receive a 0 as an input.

In [ ]:
pro_movies
sns.heatmap(pro_movies[[i for i in vocab]])

## Feature Engineering
There are several approach to take with _categorical_ values. First, one-hot encoding.

In [ ]:
# one hot enconde genres
# 